In [1]:
# Importing necessary libraries and general settings
import transformation
import compensation
import data_imports
import pandas as pd
import os
import glob
import math
import re
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from mpl_toolkits import mplot3d
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

In [2]:
dir_paraquet = 'Manufacturing_Data/'
dir_planning = 'Planing_Data/'
dir_final_save  = 'Final_df/' 

In [5]:
print('Enter the Block number, example 1030')
block = input()
block = int(block)

Enter the Block number, example 1030
1030


In [6]:
print('Enter the blade angle, muliples of 12')
angle = input()
angle = int(angle)

Enter the blade angle, muliples of 12
36


In [7]:
df_m,df_p,compensation_values_df = data_imports.data_out(block,angle,dir_paraquet,dir_planning)

In [8]:
# Just copying machine data points linear(x,y,x) and rotart (a,c) into arrays.
# These machine points (x,y,x,a,c) are transformed from machine coordinate system to workpiece coordinate system by forward tramsformation

x = df_m['MachineX'].copy(deep=True)
y = df_m['MachineY'].copy(deep=True)
z = df_m['MachineZ'].copy(deep=True)
a = np.radians(df_m['MachineA']).copy(deep=True)
c = np.radians(df_m['MachineC']).copy(deep=True)

size1 = x.shape[0]

# converting pandas series to numpy array
x = x.to_numpy()
y = y.to_numpy()
z = z.to_numpy()
a = a.to_numpy()
c = c.to_numpy()

In [9]:
#compensation_values
compensation_values = compensation_values_df.to_numpy()

In [10]:
# Within each cube we have ranges defined in x,y,z for the machine position

x_range = np.arange(-200,201,100)
y_range = np.arange(-300,301,150)
z_range = np.arange(-500,1,50)

obj3 = compensation.Compensation(compensation_values,x_range,y_range,z_range)

# Caclculation of compensation error values based on machine positions obtained through inverse transformation
deltaX1, deltaY1, deltaZ1,deltaI1,deltaJ1,deltaK1 = obj3.calculate(x,y,z)  
size3 = x.shape[0]

conc3 = np.concatenate((deltaX1.reshape(size3,1),deltaY1.reshape(size3,1),deltaZ1.reshape(size3,1),deltaI1.reshape(size3,1),deltaJ1.reshape(size3,1),deltaK1.reshape(size3,1)),axis=1)

# prininting the compensation error values
df_obj3 = pd.DataFrame(conc3, columns=['deltaX1','deltaY1','deltaZ1','deltaI1','deltaJ1','deltaK1'])
df_obj3.head(5)  
                          

,deltaX1,deltaY1,deltaZ1,deltaI1,deltaJ1,deltaK1
0,26.443495,-11.914891,-10.001572,0.0,0.0,0.0
1,26.443495,-11.914891,-10.001572,0.0,0.0,0.0
2,26.443495,-11.914891,-10.001572,0.0,0.0,0.0
3,26.443495,-11.914891,-10.001572,0.0,0.0,0.0
4,26.443495,-11.914891,-10.001572,0.0,0.0,0.0


In [11]:
x_compensated = x + deltaX1*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
y_compensated = y + deltaY1*10**-3
z_compensated = z + deltaZ1*10**-3

conc4 = np.concatenate((x_compensated.reshape(size3,1),y_compensated.reshape(size3,1),z_compensated.reshape(size3,1)),axis=1)
df_obj4 = pd.DataFrame(conc4, columns=['x_compensated','y_compensated','z_compensated'])
df_obj4.head(5)

,x_compensated,y_compensated,z_compensated
0,-158.991757,-118.613215,-79.440502
1,-158.991757,-118.613215,-79.440502
2,-158.991757,-118.613215,-79.440502
3,-158.991757,-118.613215,-79.440502
4,-158.991757,-118.613215,-79.440502


In [12]:
df_m['compensation_x'] = deltaX1*10**-3
df_m['compensation_y'] = deltaY1*10**-3
df_m['compensation_z'] = deltaZ1*10**-3
df_m.iloc[10000:10010]

,time,MachineX,MachineY,MachineZ,MachineA,MachineC,LoadSpindle,TransfX,TransfY,TransfZ,TransfI,TransfJ,TransfK,executionDuration,previousExecutionDuration,relativeTimetoPreviousCall program number,timeStamp,Tool Length,Tool Radius,G54-X,G54-Y,G54-Z,G54-A,G54-C,ProgNumber,Blade,Step,Level,Fx_mean,Fy_mean,Fz_mean,Mz_mean,Fc_mean,Fa_mean,FcN_mean,Fres_mean,Fx_sigma,Fy_sigma,Fz_sigma,Mz_sigma,Fc_sigma,Fa_sigma,FcN_sigma,Fres_sigma,Fx_max,Fy_max,Fz_max,Mz_max,Fc_max,Fa_max,FcN_max,Fres_max,compensation_x,compensation_y,compensation_z
83.810005,83.810005,-97.7303,-63.1847,-150.6453,-72.6347,8.9465,10.068182,-126.727733,176.537279,37.139965,0.169891,0.327706,-0.929379,0.001821,0.007823,0.007934,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.000000,0.000000,0.485943,0.007055,1.175781,0.000000,0.000000,0.485943,0.000000,0.000000,0.886887,0.004683,0.780559,0.000000,0.000000,0.886887,0.000000,0.000000,2.553259,0.018394,3.065742,0.000000,0.000000,2.553259,0.022540,-0.011858,-0.007957
83.815002,83.815002,-97.8991,-63.0463,-150.7641,-72.6291,8.9726,10.068182,-126.961704,176.369772,36.991380,0.159004,0.327342,-0.931432,0.005032,0.001821,0.006917,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.000000,0.000000,0.213691,0.006130,1.021649,0.000000,0.000000,0.213691,0.000000,0.000000,0.631026,0.004888,0.814599,0.000000,0.000000,0.631026,0.000000,0.000000,2.377359,0.025774,4.295648,0.000000,0.000000,2.377359,0.022534,-0.011862,-0.007955
83.825005,83.825005,-98.0677,-62.9098,-150.8837,-72.6241,8.9988,10.045455,-127.195664,176.201923,36.842323,0.148448,0.327150,-0.933240,0.004004,0.005032,0.006977,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.000000,0.000000,0.228567,0.007059,1.176531,0.000000,0.000000,0.228567,0.000000,0.000000,0.632738,0.005780,0.963416,0.000000,0.000000,0.632738,0.000000,0.000000,2.538990,0.031273,5.212174,0.000000,0.000000,2.538990,0.022528,-0.011866,-0.007953
83.835007,83.835007,-98.0677,-62.9098,-150.8837,-72.6241,8.9988,10.045455,-127.195664,176.201923,36.842323,0.148448,0.327150,-0.933240,0.007942,0.004004,0.007028,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.034709,0.000000,0.256713,0.006539,1.089781,0.034709,0.033830,0.291422,0.479683,0.000000,0.667335,0.005064,0.844080,0.479683,0.467546,0.810933,6.664060,0.000000,2.419346,0.028513,4.752179,6.664060,6.495443,6.664060,0.022528,-0.011866,-0.007953
83.840004,83.840004,-98.0677,-62.9098,-150.8837,-72.6241,8.9988,10.045455,-127.195664,176.201923,36.842323,0.148448,0.327150,-0.933240,0.001967,0.007942,0.007993,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.000000,0.000000,0.139961,0.006551,1.091837,0.000000,0.000000,0.139961,0.000000,0.000000,0.512309,0.004631,0.771906,0.000000,0.000000,0.512309,0.000000,0.000000,2.438214,0.021708,3.617969,0.000000,0.000000,2.438214,0.022528,-0.011866,-0.007953
83.845001,83.845001,-98.4038,-62.6422,-151.1252,-72.6159,9.0511,10.045455,-127.661846,175.866329,36.542546,0.128345,0.327327,-0.936154,0.006029,0.001967,0.006960,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.035661,0.042568,0.285663,0.007315,1.219202,0.078229,0.077430,0.354807,0.492840,0.588301,0.711700,0.005310,0.885014,0.765476,0.757330,1.026794,6.846851,8.173054,2.565040,0.029989,4.998133,8.173054,8.051395,8.409700,0.022515,-0.011873,-0.007948
83.855003,83.855003,-98.4038,-62.6422,-151.1252,-72.6159,9.0511,10.031818,-127.661846,175.866329,36.542546,0.128345,0.327327,-0.936154,0.002159,0.006029,0.008801,6.375063e+10,226.4591,6.0,0.0,0.0,-510.483,36.0,0.0,1030,-36,SCHLICHTEN,1,0.000000,0.068710,0.190618,0.006624,1.104018,0.068710,0.065137,0.259328,0.000000,0.774865,0.568583,0.004746,0.790929,0.774865,0.734727,0.947369,0.000000,9.122940,2.245462,0.025050,4.175060,9.122940,8.692518,9.122940,0.022515,-0.011873,-0.007948
83.865005,83.865005,-98.5532,-62.5251,-151.2339,-72.6129,9.0744,10.031818,-127.869004,175.716423,3

In [13]:
obj = transformation.Transformation(size1,angle)
# Forward Transformation fuction:
# Input : Machine points in machine coordinate system
# Output: returns too tip points and orientation in workpiece coordinate system
tool_position_workpiece_CS, tool_orientation_workpiece_CS = obj.forward(x_compensated,y_compensated,z_compensated,a,c)

X = tool_position_workpiece_CS[0,0,:]
Y = tool_position_workpiece_CS[1,0,:]
Z = tool_position_workpiece_CS[2,0,:]

I = tool_orientation_workpiece_CS[0,0,:]
J = tool_orientation_workpiece_CS[1,0,:]
K = tool_orientation_workpiece_CS[2,0,:]

# Verification the correctness of code, i.e on applying reverse transformation on the forward transformation we should get same values or equivalent values i.e (cos(a or c),sin(a or c))
machine_points_xyz, machine_direction_ac = obj.backward(X,Y,Z,I,J,K)

x_out = machine_points_xyz[0,0,:]
y_out = machine_points_xyz[1,0,:]
z_out = machine_points_xyz[2,0,:]

a_out = machine_direction_ac[0,0,:]
c_out = machine_direction_ac[1,0,:]

conc1 = np.concatenate((x.reshape(size1,1),x_out.reshape(size1,1),y.reshape(size1,1),y_out.reshape(size1,1),z.reshape(size1,1),z_out.reshape(size1,1),a.reshape(size1,1),a_out.reshape(size1,1),c.reshape(size1,1),c_out.reshape(size1,1)),axis=1)

df_obj1 = pd.DataFrame(conc1, columns=['x','x_out','y','y_out','z','z_out','a','a_out','c','c_out'])

df_obj1.iloc[10000:10005]


0 -75.2364 -75.2364 351.9877 351.9877
1 -75.2364 -75.2364 351.9877 351.9877
2 -75.2364 -75.2364 351.9877 351.9877
3 -75.2364 -75.2364 351.9877 351.9877
4 -75.2364 -75.2364 351.9877 351.9877
5 -75.2364 -75.2364 351.9877 351.9877
6 -75.2364 -75.2364 351.9877 351.9877
7 -75.2364 -75.2364 351.9877 351.9877
8 -75.2364 -75.2364 351.9877 351.9877
9 -75.2364 -75.2364 351.9877 351.9877
10 -75.2364 -75.2364 351.9877 351.9877
11 -75.2364 -75.2364 351.9877 351.9877
12 -75.2364 -75.2364 351.9877 351.9877
13 -75.2364 -75.2364 351.9877 351.9877
14 -75.2364 -75.2364 351.9877 351.9877
15 -75.2364 -75.2364 351.9877 351.9877
16 -75.2364 -75.2364 351.9877 351.9877
17 -75.2364 -75.2364 351.9877 351.9877
18 -75.2364 -75.2364 351.9877 351.9877
19 -75.2364 -75.2364 351.9877 351.9877
20 -75.2364 -75.2364 351.9877 351.9877
21 -75.2364 -75.2364 351.9877 351.9877
22 -75.2364 -75.2364 351.9877 351.9877
23 -75.2364 -75.2364 351.9877 351.9877
24 -75.2364 -75.2364 351.9877 351.9877
25 -75.2364 -75.2364 351.9877 351.9

278 -75.2364 -75.2364 351.9877 351.9877
279 -75.2364 -75.2364 351.9877 351.9877
280 -75.2364 -75.2364 351.9877 351.9877
281 -75.2364 -75.2364 351.9877 351.9877
282 -75.2364 -75.2364 351.9877 351.9877
283 -75.2364 -75.2364 351.9877 351.9877
284 -75.2364 -75.2364 351.9877 351.9877
285 -75.2364 -75.2364 351.9877 351.9877
286 -75.2364 -75.2364 351.9877 351.9877
287 -75.2364 -75.2364 351.9877 351.9877
288 -75.2364 -75.2364 351.9877 351.9877
289 -75.2364 -75.2364 351.9877 351.9877
290 -75.2364 -75.2364 351.9877 351.9877
291 -75.2364 -75.2364 351.9877 351.9877
292 -75.2364 -75.2364 351.9877 351.9877
293 -75.2364 -75.2364 351.9877 351.9877
294 -75.2364 -75.2364 351.9877 351.9877
295 -75.2364 -75.2364 351.9877 351.9877
296 -75.2364 -75.2364 351.9877 351.9877
297 -75.2364 -75.2364 351.9877 351.9877
298 -75.2364 -75.2364 351.9877 351.9877
299 -75.2364 -75.2364 351.9877 351.9877
300 -75.2364 -75.2364 351.9877 351.9877
301 -75.2364 -75.2364 351.9877 351.9877
302 -75.2364 -75.2364 351.9877 351.9877


542 -75.9438 -75.9438 367.9868 7.986800000000001
543 -75.9438 -75.9438 367.9868 7.986800000000001
544 -75.9438 -75.9438 367.9868 7.986800000000001
545 -75.9438 -75.9438 367.9868 7.986800000000001
546 -75.9438 -75.9438 367.9868 7.986800000000001
547 -75.9438 -75.9438 367.9868 7.986800000000001
548 -75.9438 -75.9438 367.9868 7.986800000000001
549 -75.9438 -75.9438 367.9868 7.986800000000001
550 -75.9438 -75.9438 367.9868 7.986800000000001
551 -75.9438 -75.9438 367.9868 7.986800000000001
552 -75.9438 -75.9438 367.9868 7.986800000000001
553 -75.9438 -75.9438 367.9868 7.986800000000001
554 -75.9438 -75.9438 367.9868 7.986800000000001
555 -75.9438 -75.9438 367.9868 7.986800000000001
556 -75.9438 -75.9438 367.9868 7.986800000000001
557 -75.9438 -75.9438 367.9868 7.986800000000001
558 -75.9438 -75.9438 367.9868 7.986800000000001
559 -75.9438 -75.9438 367.9868 7.986800000000001
560 -75.9438 -75.9438 367.9868 7.986800000000001
561 -75.9438 -75.9438 367.9868 7.986800000000001
562 -75.9438 -75.943

758 -75.9438 -75.9438 367.9868 7.986800000000001
759 -75.9438 -75.9438 367.9868 7.986800000000001
760 -75.9438 -75.9438 367.9868 7.986800000000001
761 -75.9438 -75.9438 367.9868 7.986800000000001
762 -75.9438 -75.9438 367.9868 7.986800000000001
763 -75.9438 -75.9438 367.9868 7.986800000000001
764 -75.9438 -75.9438 367.9868 7.986800000000001
765 -75.9146 -75.9146 367.9868 7.986800000000001
766 -75.9146 -75.9146 367.9868 7.986800000000001
767 -75.7944 -75.7944 367.9868 7.986800000000001
768 -75.7944 -75.7944 367.9868 7.986800000000001
769 -75.6665 -75.6665 367.9868 7.986800000000001
770 -75.6665 -75.6665 367.9868 7.986800000000001
771 -75.55470000000001 -75.55470000000001 367.9868 7.986800000000001
772 -75.55470000000001 -75.55470000000001 367.9868 7.986800000000001
773 -75.55470000000001 -75.55470000000001 367.9868 7.986800000000001
774 -75.55470000000001 -75.55470000000001 367.9868 7.986800000000001
775 -75.55470000000001 -75.55470000000001 367.9868 7.986800000000001
776 -75.5547000000

939 -72.2647 -72.2647 368.1152 8.1152
940 -72.3248 -72.3248 368.1187 8.1187
941 -72.3248 -72.3248 368.1187 8.1187
942 -72.3574 -72.3574 368.1204 8.1204
943 -72.3574 -72.3574 368.1204 8.1204
944 -72.3918 -72.3918 368.1221 8.1221
945 -72.4252 -72.4252 368.12370000000004 8.1237
946 -72.4633 -72.4633 368.1253 8.1253
947 -72.5031 -72.5031 368.127 8.127
948 -72.5444 -72.5444 368.1287 8.128700000000002
949 -72.5444 -72.5444 368.1287 8.128700000000002
950 -72.6306 -72.6306 368.1308 8.1308
951 -72.6306 -72.6306 368.1308 8.1308
952 -72.727 -72.727 368.132 8.132
953 -72.7767 -72.7767 368.1326 8.1326
954 -72.7767 -72.7767 368.1326 8.1326
955 -72.8279 -72.8279 368.1331 8.1331
956 -72.8279 -72.8279 368.1331 8.1331
957 -72.9315 -72.9315 368.1342 8.1342
958 -72.9876 -72.9876 368.1347 8.1347
959 -73.0451 -73.0451 368.1352 8.1352
960 -73.0451 -73.0451 368.1352 8.1352
961 -73.1604 -73.1604 368.136 8.136
962 -73.1604 -73.1604 368.136 8.136
963 -73.286 -73.286 368.1367 8.1367
964 -73.286 -73.286 368.1367 8

...

36954 -76.0127 -76.0127 369.1032 9.1032
36955 -76.0417 -76.0417 368.81870000000004 8.8187
36956 -76.0566 -76.0566 368.67030000000005 8.6703
36957 -76.0566 -76.0566 368.67030000000005 8.6703
36958 -76.0861 -76.0861 368.3735 8.3735
36959 -76.0861 -76.0861 368.3735 8.3735
36960 -76.0995 -76.0995 368.2375 8.2375
36961 -76.1138 -76.1138 368.08930000000004 8.0893
36962 -76.128 -76.128 367.9411 7.941099999999999
36963 -76.1687 -76.1687 367.5101 7.5100999999999996
36964 -76.2088 -76.2088 367.0689 7.0689
36965 -76.2219 -76.2219 366.9223 6.9223
36966 -76.27120000000001 -76.27120000000001 366.3519 6.3519
36967 -76.3059 -76.3059 365.9315 5.9315
36968 -76.3173 -76.3173 365.7884 5.7884
36969 -76.3286 -76.3286 365.64580000000007 5.6458
36970 -76.3286 -76.3286 365.64580000000007 5.6458
36971 -76.3496 -76.3496 365.3741 5.3741
36972 -76.3496 -76.3496 365.3741 5.3741
36973 -76.3705 -76.3705 365.0932 5.0932
36974 -76.3705 -76.3705 365.0932 5.0932
36975 -76.3906 -76.3906 364.8148 4.8148
36976 -76.3906 -76.

37277 -74.4998 -74.4998 375.82140000000004 15.8214
37278 -74.4998 -74.4998 375.82140000000004 15.8214
37279 -74.4976 -74.4976 375.8928 15.892800000000001
37280 -74.4953 -74.4953 375.97080000000005 15.9708
37281 -74.4931 -74.4931 376.0479 16.0479
37282 -74.4909 -74.4909 376.1243 16.1243
37283 -74.4909 -74.4909 376.1243 16.1243
37284 -74.4888 -74.4888 376.2003 16.200300000000002
37285 -74.487 -74.487 376.2684 16.2684
37286 -74.4851 -74.4851 376.3439 16.3439
37287 -74.4833 -74.4833 376.4191 16.4191
37288 -74.4816 -74.4816 376.494 16.494
37289 -74.48 -74.48 376.56850000000003 16.5685
37290 -74.4758 -74.4758 376.78220000000005 16.7822
37291 -74.4746 -74.4746 376.855 16.855
37292 -74.4746 -74.4746 376.855 16.855
37293 -74.47350000000002 -74.47350000000002 376.92740000000003 16.9274
37294 -74.4707 -74.4707 377.1342 17.1342
37295 -74.4699 -74.4699 377.20439999999996 17.2044
37296 -74.4699 -74.4699 377.20439999999996 17.2044
37297 -74.4688 -74.4688 377.336 17.336
37298 -74.4679 -74.4679 377.471

37571 -75.7651 -75.7651 358.9864 358.9864
37572 -75.7651 -75.7651 358.9864 358.9864
37573 -75.74 -75.74 359.1038 359.1038
37574 -75.7281 -75.7281 359.1625 359.1625
37575 -75.7157 -75.7157 359.2261 359.2261
37576 -75.7041 -75.7041 359.2875 359.2875
37577 -75.6926 -75.6926 359.35010000000005 359.35010000000005
37578 -75.6809 -75.6809 359.4168000000001 359.4168000000001
37579 -75.6588 -75.6588 359.5481 359.5481
37580 -75.6588 -75.6588 359.5481 359.5481
37581 -75.63710000000002 -75.63710000000002 359.68580000000003 359.68580000000003
37582 -75.63710000000002 -75.63710000000002 359.68580000000003 359.68580000000003
37583 -75.6162 -75.6162 359.8265 359.8265
37584 -75.6162 -75.6162 359.8265 359.8265
37585 -75.5937 -75.5937 359.9884 359.9884
37586 -75.5793 -75.5793 360.0984 0.0984
37587 -75.5652 -75.5652 360.2101 0.2101
37588 -75.5652 -75.5652 360.2101 0.2101
37589 -75.5516 -75.5516 360.3236 0.3236
37590 -75.5516 -75.5516 360.3236 0.3236
37591 -75.5384 -75.5384 360.43870000000004 0.4387
37592 

37844 -75.2364 -75.2364 363.9877 3.9877
37845 -75.2364 -75.2364 363.9877 3.9877
37846 -75.2364 -75.2364 363.9877 3.9877
37847 -75.2364 -75.2364 363.9877 3.9877
37848 -75.2364 -75.2364 363.9877 3.9877
37849 -75.2364 -75.2364 363.9877 3.9877
37850 -75.2364 -75.2364 363.9877 3.9877
37851 -75.2364 -75.2364 363.9877 3.9877
37852 -75.2364 -75.2364 363.9877 3.9877
37853 -75.2364 -75.2364 363.9877 3.9877
37854 -75.2364 -75.2364 363.9877 3.9877
37855 -75.2364 -75.2364 363.9877 3.9877
37856 -75.2364 -75.2364 363.9877 3.9877
37857 -75.2364 -75.2364 363.9877 3.9877
37858 -75.2364 -75.2364 363.9877 3.9877
37859 -75.2364 -75.2364 363.9877 3.9877
37860 -75.2364 -75.2364 363.9877 3.9877
37861 -75.2364 -75.2364 363.9877 3.9877
37862 -75.2364 -75.2364 363.9877 3.9877
37863 -75.2364 -75.2364 363.9877 3.9877
37864 -75.2364 -75.2364 363.9877 3.9877
37865 -75.2364 -75.2364 363.9877 3.9877
37866 -75.2364 -75.2364 363.9877 3.9877
37867 -75.2364 -75.2364 363.9877 3.9877
37868 -75.2364 -75.2364 363.9877 3.9877


,x,x_out,y,y_out,z,z_out,a,a_out,c,c_out
10000,-97.7303,-97.707760,-63.1847,-63.196558,-150.6453,-150.653257,-1.267715,-1.267715,0.156146,6.439331
10001,-97.8991,-97.876566,-63.0463,-63.058162,-150.7641,-150.772055,-1.267617,-1.267617,0.156601,6.439787
10002,-98.0677,-98.045172,-62.9098,-62.921666,-150.8837,-150.891653,-1.267530,-1.267530,0.157059,6.440244
10003,-98.0677,-98.045172,-62.9098,-62.921666,-150.8837,-150.891653,-1.267530,-1.267530,0.157059,6.440244
10004,-98.0677,-98.045172,-62.9098,-62.921666,-150.8837,-150.891653,-1.267530,-1.267530,0.157059,6.440244


In [ ]:
from scipy.optimize import minimize,NonlinearConstraint

def fun(x):
    return -np.sin(x[0])*np.cos(x[1]), np.cos(x[0])

nonlinear_constraint = NonlinearConstraint(fun, np.array([J[0],K[0]]),np.array([J[0],K[0]]))

def objective_function(x,I):
    return (I - (np.sin(x[0]))*np.sin(x[1]) )

residual1 = minimize(objective_function, np.array([-0.1,0.1]), method='trust-constr',constraints=[nonlinear_constraint], args=(I[0]))
#residual1 = minimize(objective_function, np.array([-0.1,0.1]), method='SLSQP',constraints=[nonlinear_constraint], args=(I[0]))
residual1

In [ ]:
from scipy.optimize import shgo

def objective_function(x,I):
    return (I - (np.sin(x[0]))*np.sin(x[1]) )

eq_constraints =  { 'type': 'eq',
                     'fun': lambda x: np.array([J[0]+np.sin(x[0])*np.cos(x[1]),
                                                K[0]-np.cos(x[0])]),
                     'args': [J,K]
                    }
residual2 = shgo(objective_function,constraints=[eq_constraints], args=[I[0]], bounds=([-np.pi,np.pi],[3*np.pi,3*np.pi]),sampling_method='simplicial')
residual2

In [ ]:
from scipy.optimize import root
    
def inv_angles(x, I, J ,K):
    f = [ (I - (np.sin(x[0])*np.sin(x[1]))),
          (J + (np.sin(x[0])*np.cos(x[1]))),
          (K - np.cos(x[0]))
        ]
    return f

for i in np.arange(len(I)):
    roots[i] = root(inv_angles,[-1,6], method='lm', args=(I[i],J[i],K[i]), options={'xtol': 1.49012e-15, 'ftol': 1.49012e-15, 'factor':1}) 
    print(i, np.rad2deg(roots[i]['x'][0]),np.rad2deg(a[i]),np.rad2deg(roots[i]['x'][1])+angle,np.rad2deg(c[i]))

In [20]:
conc11 = np.concatenate((x.reshape(size1,1),y.reshape(size1,1),z.reshape(size1,1),a.reshape(size1,1),c.reshape(size1,1),X.reshape(size1,1),Y.reshape(size1,1),Z.reshape(size1,1),I.reshape(size1,1),J.reshape(size1,1),K.reshape(size1,1)),axis=1)
dfout11 = pd.DataFrame(conc11, columns=['x','y','z','a','c','X','Y','Z','I','J','K'])
dfout11.head(10)


ValueError: cannot reshape array of size 1 into shape (37971,1)

In [ ]:
# example
n = 15000
distance1 = (((df_p['Tool Tip Point X']-X[n])**2+(df_p['Tool Tip Point Y']-Y[n])**2+(df_p['Tool Tip Point Z']-Z[n])**2)**(1/2))
distance1.min()

In [ ]:
tool_tip_X = df_p['Tool Tip Point X'].to_numpy()
tool_tip_Y = df_p['Tool Tip Point Y'].to_numpy()
tool_tip_Z = df_p['Tool Tip Point Z'].to_numpy()
X_inv=tool_tip_X
Y_inv=tool_tip_Y
Z_inv=tool_tip_Z

In [ ]:
# Plotting the result obtained during the study
def subplotter(low,high,X,Y,Z,X_inv,Y_inv,Z_inv):
    
    range = np.arange(low,high)
    fig,axs = plt.subplots(3,figsize=(10,10))

    axs[0].plot(range,X[range],label = 'X', color='blue')
    axs[0].plot(range,X_inv[range],label = 'X_inv', color='red')
    axs[0].legend() 
    #axs[0].set_title('Toot Path')   
    
    axs[1].plot(range,Y[range], label = 'Y', color='blue')
    axs[1].plot(range,Y_inv[range], label = 'Y_inv', color='red') 
    axs[1].legend()
    #axs[1].set_title('CosA')

    axs[2].plot(range,Z[range], label = 'Y', color='blue')
    axs[2].plot(range,Z_inv[range] , label = 'Y_inv', color='red')
    axs[2].legend()
    #axs[2].set_title('SinC')
    
    fig.canvas.draw()
    fig.canvas.flush_events()


In [ ]:
subplotter(0,df_p.shape[0],X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
subplotter(10000,35000,X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
subplotter(20000,21000,X,Y,Z,X_inv,Y_inv,Z_inv)

In [ ]:
# Best Version
lst1 = []
lst2 = []

dist  = np.zeros(len(tool_tip_X))
print('Enter the tolerance number value, example 3 mm')
tol = input()

count = 0 
k = 5

for i in np.arange(len(X)):   # 47917
    
    dist = (((tool_tip_X-X[i])**2+(tool_tip_Y-Y[i])**2+(tool_tip_Z-Z[i])**2)**(1/2))
    
    pos = np.argpartition(dist, k)
    pos = pos[:k]
    min_val = dist[pos]
    #print(pos,min_val)
    
    pos = pos[np.argmin(abs(pos - i))]
    min_val = dist[pos]
    
    #print(i,pos,min_val)
    if(abs(min_val<=float(tol))): 
        lst1.append(pos)
        lst2.append(min_val)
        count = count+1
        print(i,pos,min_val,count)
    else:
        lst1.append(np.nan)
        lst2.append(np.nan)
   

In [ ]:
print(np.shape(lst1))
#np.array(lst1)

In [ ]:
(unique,count) = np.unique(lst1, return_counts=True)

df_freq = pd.DataFrame({'Unique Cluster Points':np.array(unique),'Count':np.array(count)})
df_freq = df_freq.dropna()
df_freq.iloc[:]

In [ ]:
df_cluster = pd.DataFrame({'cluster index in planning data':np.array(lst1),'distance error':np.array(lst2)})
#df_cluster = df_cluster.dropna()
df_cluster.iloc[22000:22100]

In [ ]:
# Clustering
lst3 = []
tcp_val = []
for i in np.arange(len(tool_tip_X)):     # each planning point
    
    matching = np.where(np.array(lst1)==i)    # see what all machine points matches
    tcp_val.append([lst2[index] for index in matching[0]])          # get the same minmum tcp error distance 
    lst3.append(matching[0])
    print(i,matching[0])
    

In [ ]:
# list for plotting
pltlst = []
for i in np.arange(len(tool_tip_X)):
    if (len(lst3[i]) != 0):
        pltlst.append(i)
        #print(i,lst3[i])

In [ ]:
# Plot to show clusters
xdata, ydata, zdata = np.zeros(len(pltlst),),np.zeros(len(pltlst),),np.zeros(len(pltlst),)

plt.figure(figsize=(10,15))
ax = plt.axes(projection='3d')
ax.set_zlim3d([0,30])
ax.set_xlabel('$X$', fontsize=20)

ax.set_ylabel('$Y$', fontsize=20)
ax.set_zlabel('$Z$', fontsize=20)

for i in np.arange(len(pltlst)):
    xdata[i] = tool_tip_X[pltlst[i]]
    ydata[i] = tool_tip_Y[pltlst[i]]
    zdata[i] = tool_tip_Z[pltlst[i]]

ax.scatter3D(xdata, ydata, zdata, color = 'red',label = 'Planning Data')
ax.legend()

xdata2, ydata2, zdata2 = np.zeros(len(pltlst),),np.zeros(len(pltlst),),np.zeros(len(pltlst),)

for i in np.arange(len(pltlst)):
    xdata2[i] = np.average(X[tuple([lst3[pltlst[i]]])])
    ydata2[i] = np.average(Y[tuple([lst3[pltlst[i]]])])
    zdata2[i] = np.average(Z[tuple([lst3[pltlst[i]]])])

ax.scatter3D(xdata2, ydata2, zdata2, color = 'blue',label = 'Machine Data Avg')
ax.legend()

In [ ]:
num = -10
xdata, ydata, zdata = np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),)

plt.figure(figsize=(6,6))
ax = plt.axes(projection='3d')

ax.scatter3D(tool_tip_X[pltlst[num]],tool_tip_Y[pltlst[num]] , tool_tip_Z[pltlst[num]])

for i in np.arange(len(lst3[pltlst[num]])):
    xdata[i] = X[lst3[pltlst[num]]][i]
    ydata[i] = Y[lst3[pltlst[num]]][i]
    zdata[i] = Z[lst3[pltlst[num]]][i]
    
ax.scatter3D(xdata, ydata, zdata)

In [ ]:
#for num in np.arange(len(pltlst)):
print('Input two numbers between 0 and {} with small difference for plots exaple 10000 then 10010'.format(len(pltlst)))
lim1 = input()
lim2 = input()

for num in np.arange(len(pltlst[int(lim1):int(lim2)])):
    xdata, ydata, zdata = np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),),np.zeros(len(lst3[pltlst[num]]),)
    
    fig = plt.figure(figsize=(6,6))
    print(num)
    ax = plt.axes(projection='3d')

    ax.scatter3D(tool_tip_X[pltlst[num]],tool_tip_Y[pltlst[num]] , tool_tip_Z[pltlst[num]])

    for i in np.arange(len(lst3[pltlst[num]])):
        xdata[i] = X[lst3[pltlst[num]]][i]
        ydata[i] = Y[lst3[pltlst[num]]][i]
        zdata[i] = Z[lst3[pltlst[num]]][i]
    
    ax.scatter3D(xdata, ydata, zdata)
    plt.pause(0.1)
   

In [ ]:
lst4 = []
for i in np.arange(len(lst3)):
    lst4.append(df_m.iloc[list(lst3[i])].mean(axis=0))
    
mean_m = pd.concat(lst4,axis=1).T
mean_m.iloc[-100:-1]

In [ ]:
tcp_avg = [(lambda x: sum(x)/len(x))(item) if len(item)!=0 else np.nan for item in tcp_val]
len(tcp_avg)
#tcp_avg

In [ ]:
final_df = pd.concat([df_p,mean_m, pd.DataFrame({'tcp_error':tcp_avg})], axis=1)
final_df = final_df.drop(['Level','Step'],axis=1)
final_df = final_df.dropna()
print(final_df.shape)
final_df.iloc[0:10]

In [ ]:
final_df.shape 

### Using Combined dataset

In [ ]:
x_final = final_df['MachineX'].copy(deep=True)                 # using machine data from newly created combined dataset
y_final = final_df['MachineY'].copy(deep=True)
z_final = final_df['MachineZ'].copy(deep=True)
a_final = final_df['MachineA'].copy(deep=True)
c_final = final_df['MachineC'].copy(deep=True)

size1_final = x_final.shape[0]

# converting pandas series to numpy array
x_final = x_final.to_numpy()
y_final = y_final.to_numpy()
z_final = z_final.to_numpy()
a_final = a_final.to_numpy(dtype =  np.float64)
a_final = np.deg2rad(a_final)
c_final = c_final.to_numpy(dtype =  np.float64)
c_final = np.deg2rad(c_final)

tool_tip_X_final = final_df['Tool Tip Point X'].to_numpy()        # using newly created combined dataset
tool_tip_Y_final = final_df['Tool Tip Point Y'].to_numpy()
tool_tip_Z_final = final_df['Tool Tip Point Z'].to_numpy()
X_inv_final = tool_tip_X_final
Y_inv_final = tool_tip_Y_final
Z_inv_final = tool_tip_Z_final

In [ ]:
#compensation_values
compensation_values = compensation_values_df.to_numpy()

# Within each cube we have ranges defined in x,y,z for the machine position

x_range = np.arange(-200,201,100)
y_range = np.arange(-300,301,150)
z_range = np.arange(-500,1,50)

obj3_final = compensation.Compensation(compensation_values,x_range,y_range,z_range)

# Caclculation of compensation error values based on machine positions obtained through inverse transformation
deltaX1_final, deltaY1_final, deltaZ1_final,deltaI1_final,deltaJ1_final,deltaK1_final = obj3_final.calculate(x_final,y_final,z_final)  
size3_final = x_final.shape[0]
conc3_final = np.concatenate((deltaX1_final.reshape(size3_final,1),deltaY1_final.reshape(size3_final,1),deltaZ1_final.reshape(size3_final,1),deltaI1_final.reshape(size3_final,1),deltaJ1_final.reshape(size3_final,1),deltaK1_final.reshape(size3_final,1)),axis=1)

# prininting the compensation error values
df_obj3_final = pd.DataFrame(conc3_final, columns=['deltaX1 final','deltaY1 final','deltaZ1 final','deltaI1 final','deltaJ1 final','deltaK1 final'])
df_obj3_final.head(5)  

x_compensated_final = x_final + deltaX1_final*10**-3    # since given compensation is to be converted from microns to mm (10**-6 x 10**3 = 10**-3)
y_compensated_final = y_final + deltaY1_final*10**-3
z_compensated_final = z_final + deltaZ1_final*10**-3

conc4_final = np.concatenate((x_compensated_final.reshape(size3_final,1),y_compensated_final.reshape(size3_final,1),z_compensated_final.reshape(size3_final,1)),axis=1)
df_obj4_final = pd.DataFrame(conc4_final, columns=['x_compensated final','y_compensated final','z_compensated final'])
df_obj4_final.head(5)
                          


In [ ]:
obj_final = transformation.Transformation(size1_final,angle)
# Forward Transformation fuction:
# Input : Machine points in machine coordinate system
# Output: returns too tip points and orientation in workpiece coordinate system
tool_position_workpiece_CS_final, tool_orientation_workpiece_CS_final = obj_final.forward(x_compensated_final,y_compensated_final,z_compensated_final,a_final,c_final)

X_final = tool_position_workpiece_CS_final[0,0,:]
Y_final = tool_position_workpiece_CS_final[1,0,:]
Z_final = tool_position_workpiece_CS_final[2,0,:]

I_final = tool_orientation_workpiece_CS_final[0,0,:]
J_final = tool_orientation_workpiece_CS_final[1,0,:]
K_final = tool_orientation_workpiece_CS_final[2,0,:]

# Verification the correctness of code, i.e on applying reverse transformation on the forward transformation we should get same values
machine_points_xyz_final, machine_direction_ac_final = obj_final.backward(X_final,Y_final,Z_final,I_final,J_final,K_final)

x_out_final = machine_points_xyz_final[0,0,:]
y_out_final = machine_points_xyz_final[1,0,:]
z_out_final = machine_points_xyz_final[2,0,:]

a_out_final = machine_direction_ac_final[0,0,:]
c_out_final = machine_direction_ac_final[1,0,:]

conc1_final = np.concatenate((x_final.reshape(size1_final,1),x_out_final.reshape(size1_final,1),y_final.reshape(size1_final,1),y_out_final.reshape(size1_final,1),z_final.reshape(size1_final,1),z_out_final.reshape(size1_final,1),a_final.reshape(size1_final,1),a_out_final.reshape(size1_final,1),c_final.reshape(size1_final,1),c_out_final.reshape(size1_final,1)),axis=1)
df_obj1_final = pd.DataFrame(conc1_final, columns=['x final','x_out final','y final','y_out final','z final','z_out final','a final','a_out final','c final','c_out'])

df_obj1_final.head(2)

In [ ]:
conc11_final = np.concatenate((x_final.reshape(size1_final,1),y_final.reshape(size1_final,1),z_final.reshape(size1_final,1),a_final.reshape(size1_final,1),c_final.reshape(size1_final,1),X_final.reshape(size1_final,1),Y_final.reshape(size1_final,1),Z_final.reshape(size1_final,1),I_final.reshape(size1_final,1),J_final.reshape(size1_final,1),K_final.reshape(size1_final,1)),axis=1)
dfout11_final = pd.DataFrame(conc11_final, columns=['x final','y final','z final','a final','c final','X final','Y final','Z final','I final','J final','K final'])
dfout11_final.head(2)


In [ ]:
subplotter(0,final_df.shape[0],X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)   # using combined dataset

In [ ]:
final_df.shape[0]

In [ ]:
subplotter(0,21500,X_final,Y_final,Z_final,X_inv_final,Y_inv_final,Z_inv_final)   # using combined dataset

In [ ]:
plt.plot((final_df['tcp_error']).iloc[1000:3500])
plt.title('TCP Error plot')

In [ ]:
mean = final_df['tcp_error'].mean()
mean

In [ ]:
median = final_df['tcp_error'].median()
median

In [ ]:
minimum = final_df['tcp_error'].min()
minimum

In [ ]:
maximum = final_df['tcp_error'].max()  
maximum

In [ ]:
final_df = final_df.iloc[0:21500]

In [ ]:
final_df.to_excel(str(dir_final_save)+'finaldf_forward_with_compensation'+str(block)+'__'+str(angle)+'.xlsx') 